# 파이썬으로 풀어보는 수학

## 6장 기하학적 형상과 프랙탈 그리기

### 1. MatPlotLib의 패치(patches)로 기하학적 형상 그리기

앞장에서 `matplotlib`을 사용하여 그래프를 그리는 것은 많이 해봤습니다.

In [1]:
import matplotlib.pyplot as plt

x = [1, 2, 3]
y = [1, 2, 3]

plt.plot(x, y)
plt.show()

![그래프](./image/DoingMathWithPython.Ch06.01.png)

우리가 명시적으로 생성하지는 않았지만 그래프를 위해 `Figure`객체와 `Axes`객체를 내부적으로 자동생성하여서 그 위에 그래프를 작성한 것입니다.
명시적으로 `Figure`와 `Axes`객체를 생성 할 수 있습니다.

In [3]:
fig = plt.figure()
fig

In [4]:
ax = plt.axes()
ax

명시적 생성뿐 아니라 현재 객체에 대한 참조를 가져오는 함수도 존재합니다.
만약 해당 객체가 존재하지 않는 상태에서는 생성을 합니다.

In [5]:
plt.gcf()

In [6]:
plt.gca()

이 얘기를 한 이유는 `matplotlib`에 도형(기하학적 형상)을 그리는데 필요하기 때문입니다.

#### 원 그리기

원을 생성하여 축(axes)에 추가하는 방법으로 그릴 수 있습니다.

In [12]:
circle = plt.Circle((0, 0), radius = 0.5)
plt.gca().add_patch(circle)
plt.axis('scaled')
plt.show()

![원](./image/DoingMathWithPython.Ch06.02.png)

패치의 색상을 변경하려면 `fc`(채우기)와 `ec`(테두리)에 색상을 전달하면 됩니다.
원(`Circle`) 이외에 타원(`Ellipse`), 다각형(`Polygon`), 직사각형(`Rectangle`) 등의 여러가지 다른 패치도 지원합니다.

In [20]:
circle = plt.Rectangle((1,1), height = 3, width = 5, fc = 'r', ec = 'g')
plt.gca().add_patch(circle)
plt.axis('scaled')
plt.show()

![사각형](./image/DoingMathWithPython.Ch06.03.png)

### 2. 움직이는 그림 만들기

`matplotlib`의 `animation`기능을 이용하면 움직이는 그림을 만들 수 있습니다.
윈도우를 닫을때까지 반지름이 무한대로 커지는 원을 그려보겠습니다.

In [25]:
from matplotlib import animation
from matplotlib import pyplot as plt

circle = plt.Circle((0,0), 0.05)

def update_radius(i, circle):
    circle.radius = i * 0.5
    return circle

fig = plt.gcf()

ax = plt.axes(xlim=(-10,10), ylim=(-10,10))
ax.set_aspect('equal')
ax.add_patch(circle)

ani = animation.FuncAnimation(fig, update_radius, fargs = (circle,), frames = 30, interval=50)

plt.title('Simple Circle Animation')
plt.show()

![원 애니메이션](./image/DoingMathWithPython.Ch06.04.png)

중심에서부터 원이 점점 커지는 애니메이션이 30 프레임만큼 진행된 후 후 다시 처음부터 시작하는 것을 확인할 수 있습니다.  
`animation.FuncAnimation`에 전달되는 인자들의 목록을 살펴보겠습니다.

- `fig` : 현 그림의 객체
- `update_radius` : 프레임 번호와 프레임마다 갱신하기를 바라는 패치 객체가 전달되어서 원하는 모양으로 변경한 뒤 패치 객체를 리턴
- `fargs` : `update_radius()`에게 전달되는 인자들의 목록 (프레임 번호는 제외)
- `frames` : 애니메이션의 프레임 개수. 여기에 적힌 수 만큼 `update_radius()`를 호출하게 됨
- `interval` : 프레임 간의 시간 간격 (밀리미터 단위)
- `repeat` : `True`, `False` 값을 가질 수 있으며 애니메이션의 반복 유무를 결정. (default는 `True`)

`animation.FuncAnimation` 객체를 사용하지 않으면서 변수로 저장한 이유는 가비지 컬렉터에 의해서 해제되지 않게하기 위함입니다.

#### 투사체의 포물선 애니메이션

2장에서 던진 공에 대한 포물선 운동 궤적을 그래프로 표시해보았습니다.
이번에는 이를 애니메이션으로 구현해 보겠습니다.

먼저 2장에서 배운 내용에 대해서 간략하게 복습해 보겠습니다.

- 초기속도 : u
- 초기던진 각도 : theta
- x방향 속도 : u * cos(theta)
- y방향 속도 : u * sin(theta) - gt

비행시간은 y방향으로 최고점에 도달하는 시간의 2배입니다.
최고점에 도달하는 시간은 y방향의 속도가 0이 되는 시점입니다.
이를 `sympy`를 이용해서 풀어보겠습니다.

In [34]:
from sympy import Symbol, symbols, sin, cos, solve, pprint
u, theta, g, time = symbols('u,theta,g,t')

xSpeed = u * cos(theta)
ySpeed = u * sin(theta) - g * time
pprint(ySpeed)

-g⋅t + u⋅sin(θ)


In [35]:
s = solve(ySpeed, time)
pprint(s[0] * 2)

2⋅u⋅sin(θ)
──────────
    g     


- 비행시간 : 2 * u * sin(theta) / g

이제 필요한 수식은 다 나왔습니다.
애니메이션을 실행하기 전에 위 수식이 맞는지 확인하기 위해 포물선을 한번 그려보겠습니다.

In [38]:
xDistance = xSpeed * time
pprint(xDistance)

t⋅u⋅cos(θ)


In [41]:
from sympy import expand
yDistance = expand(ySpeed * time)
pprint(yDistance)

     2             
- g⋅t  + t⋅u⋅sin(θ)
